# NumPy views and copies

- Operations that only require changing the metadata always do so, and return a **view**
- Operations that cannot be executed by changing the metadata create a new memory block, and return a **copy**

In [13]:
import numpy as np


def print_info(a):
    """ Print the content of an array, and its metadata. """
    
    txt = f"""
dtype\t{a.dtype}
ndim\t{a.ndim}
shape\t{a.shape}
strides\t{a.strides}
    """

    print(a)
    print(txt)


In [32]:
x = np.arange(12).reshape(3, 4).copy()
print_info(x)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    


# Views

Operations that only require changing the metadata always do so, and return a **view**

In [33]:
y = x[0::2, 1::2]
print_info(y)

[[ 1  3]
 [ 9 11]]

dtype	int64
ndim	2
shape	(2, 2)
strides	(64, 16)
    


A view shares the same memory block as the original array. 

CAREFUL: Modifying the view changes the original array and all an other views of that array as well!

In [34]:
z = x.reshape(1, 12)
print_info(z)

[[ 0  1  2  3  4  5  6  7  8  9 10 11]]

dtype	int64
ndim	2
shape	(1, 12)
strides	(96, 8)
    


In [35]:
y += 100
print_info(y)

[[101 103]
 [109 111]]

dtype	int64
ndim	2
shape	(2, 2)
strides	(64, 16)
    


In [37]:
print_info(x)
print_info(z)

[[  0 101   2 103]
 [  4   5   6   7]
 [  8 109  10 111]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    
[[  0 101   2 103   4   5   6   7   8 109  10 111]]

dtype	int64
ndim	2
shape	(1, 12)
strides	(96, 8)
    


Functions that take an array as an input should avoid modifying it in place! 

Always make a copy or be super extra clear in the docstring.

In [45]:
def robust_log(a, cte=1e-10):
    """ Returns the log of an array, avoiding troubles when a value is 0.
    
    Add a tiny constant to the values of `a` so that they are not 0. 
    `a` is expected to have non-negative values.
    """
    a[a == 0] += cte
    return np.log(a)
    


In [57]:
a = np.array([[0.3, 0.01], [0, 1]])
np.log(a)

/tmp/ipykernel_48764/1018405258.py:2: RuntimeWarning: divide by zero encountered in log
  np.log(a)


array([[-1.2039728 , -4.60517019],
       [       -inf,  0.        ]])

In [58]:
# This is a view of `a`
b = a[1, :]
print_info(b)

[0. 1.]

dtype	float64
ndim	1
shape	(2,)
strides	(8,)
    


In [59]:
robust_log(a)

array([[ -1.2039728 ,  -4.60517019],
       [-23.02585093,   0.        ]])

In [60]:
a

array([[3.e-01, 1.e-02],
       [1.e-10, 1.e+00]])

In [61]:
b

array([1.e-10, 1.e+00])

Better to make a copy!

In [62]:
def robust_log(a, cte=1e-10):
    """ Returns the log of an array, avoiding troubles when a value is 0.
    
    Add a tiny constant to the values of `a` so that they are not 0. 
    `a` is expected to have non-negative values.
    """
    a = a.copy()
    a[a == 0] += cte
    return np.log(a)

In [66]:
a = np.array([[0.3, 0.01], [0, 1]])
b = a[1, :]

robust_log(a)

array([[ -1.2039728 ,  -4.60517019],
       [-23.02585093,   0.        ]])

In [67]:
a

array([[0.3 , 0.01],
       [0.  , 1.  ]])

In [68]:
b

array([0., 1.])

# Copies

Operations that cannot be executed by changing the metadata create a new memory block, and return a **copy**

In [72]:
x = np.arange(12).reshape(3, 4).copy()
print_info(x)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    


Choosing row, columns, or individual elements of an array by giving explicitly their indices (a.k.a "fancy indexing") it's an operation that in general cannot be executed by changing the metadata alone.

Therefore, **fancy indexing always returns a copy**.

In [77]:
# Get the first and second column
y = x[:, [0, 1]]
print_info(y)

[[0 1]
 [4 5]
 [8 9]]

dtype	int64
ndim	2
shape	(3, 2)
strides	(8, 24)
    


In [79]:
y += 1000
print_info(y)
# the original array is unchanged => not a view!
print_info(x)

[[2000 2001]
 [2004 2005]
 [2008 2009]]

dtype	int64
ndim	2
shape	(3, 2)
strides	(8, 24)
    
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    


In [80]:
y = x[[0, 0, 2], [1, 0, 3]]
print_info(y)

[ 1  0 11]

dtype	int64
ndim	1
shape	(3,)
strides	(8,)
    


In [81]:
y += 1000
print_info(y)
# the original array is unchanged => not a view!
print_info(x)

[1001 1000 1011]

dtype	int64
ndim	1
shape	(3,)
strides	(8,)
    
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

dtype	int64
ndim	2
shape	(3, 4)
strides	(32, 8)
    


Any operation that computes new values also returns a copy.

In [82]:
y = x * 7.1
print_info(y)

[[ 0.   7.1 14.2 21.3]
 [28.4 35.5 42.6 49.7]
 [56.8 63.9 71.  78.1]]

dtype	float64
ndim	2
shape	(3, 4)
strides	(32, 8)
    
